In [ ]:
import struct

class SHA256:
    
    # 64 hằng số K (32-bit) lấy từ căn bậc 3 của 64 số nguyên tố đầu tiên
    _K = (
        0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
        0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
        0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
        0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
        0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
        0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
        0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
        0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
    )

    # 8 giá trị Hash ban đầu (H0 -> H7) lấy từ căn bậc 2 của 8 số nguyên tố đầu tiên
    _H_INITIAL = (
        0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
        0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
    )

    def __init__(self):
        # Copy giá trị khởi tạo vào biến instance để xử lý (tránh sửa hằng số gốc)
        self._h = list(self._H_INITIAL)



    @staticmethod
    def _rotr(x: int, n: int) -> int:
        """Xoay phải n bit (Rotate Right)"""
        return ((x >> n) | (x << (32 - n))) & 0xFFFFFFFF

    @staticmethod
    def _shr(x: int, n: int) -> int:
        """Dịch phải n bit (Shift Right)"""
        return (x >> n) & 0xFFFFFFFF

    @staticmethod
    def _ch(x: int, y: int, z: int) -> int:
        """Hàm Chọn (Choose): (x AND y) XOR (NOT x AND z)"""
        return (x & y) ^ (~x & z)

    @staticmethod
    def _maj(x: int, y: int, z: int) -> int:
        """Hàm Đa số (Majority): (x AND y) XOR (x AND z) XOR (y AND z)"""
        return (x & y) ^ (x & z) ^ (y & z)

    @staticmethod
    def _sigma0(x: int) -> int:
        """Hàm Sigma lớn 0 (Dùng trong vòng lặp nén)"""
        # ROTR(2) ^ ROTR(13) ^ ROTR(22)
        return SHA256._rotr(x, 2) ^ SHA256._rotr(x, 13) ^ SHA256._rotr(x, 22)

    @staticmethod
    def _sigma1(x: int) -> int:
        """Hàm Sigma lớn 1 (Dùng trong vòng lặp nén)"""
        # ROTR(6) ^ ROTR(11) ^ ROTR(25)
        return SHA256._rotr(x, 6) ^ SHA256._rotr(x, 11) ^ SHA256._rotr(x, 25)

    @staticmethod
    def _gamma0(x: int) -> int:
        """Hàm Sigma nhỏ 0 (Dùng để mở rộng tin nhắn)"""
        # ROTR(7) ^ ROTR(18) ^ SHR(3)
        return SHA256._rotr(x, 7) ^ SHA256._rotr(x, 18) ^ SHA256._shr(x, 3)

    @staticmethod
    def _gamma1(x: int) -> int:
        """Hàm Sigma nhỏ 1 (Dùng để mở rộng tin nhắn)"""
        # ROTR(17) ^ ROTR(19) ^ SHR(10)
        return SHA256._rotr(x, 17) ^ SHA256._rotr(x, 19) ^ SHA256._shr(x, 10)



    def _pad_message(self, message: bytes) -> bytes:
        """Bước 1: Đệm dữ liệu (Padding)"""
        # Lấy độ dài tin nhắn gốc (tính bằng bit)
        original_bit_len = len(message) * 8
        
        # Thêm bit '1' (byte 0x80 = 10000000)
        message += b'\x80'
        
        # Thêm bit '0' cho đến khi (len % 64) == 56 bytes (448 bits)
        while (len(message) % 64) != 56:
            message += b'\x00'
            
        # Thêm độ dài gốc (64-bit Big Endian) vào cuối
        # struct.pack('>Q') tương đương: convert số sang 8 bytes, Big Endian
        message += struct.pack('>Q', original_bit_len)
        
        return message

    def _prepare_message_schedule(self, chunk: bytes) -> list[int]:
        """Bước 2: Tạo Lịch trình tin nhắn (Mở rộng từ 16 từ -> 64 từ)"""
        w = [0] * 64
        
        # 16 từ đầu: Copy từ dữ liệu gốc
        for i in range(16):
            # Cắt 4 bytes, chuyển thành số nguyên (Big Endian)
            w[i] = int.from_bytes(chunk[i*4 : (i+1)*4], 'big')
            
        # 48 từ sau: Tính toán mở rộng
        for i in range(16, 64):
            s1 = self._gamma1(w[i-2])
            s0 = self._gamma0(w[i-15])
            # w[i] = w[i-16] + s0 + w[i-7] + s1 (Cộng modulo 2^32)
            w[i] = (w[i-16] + s0 + w[i-7] + s1) & 0xFFFFFFFF
            
        return w

    def _compress_chunk(self, chunk: bytes):
        """Bước 3: Vòng lặp nén (Compression Loop)"""
        # Chuẩn bị 64 từ dữ liệu
        w = self._prepare_message_schedule(chunk)
        
        # Lấy trạng thái băm hiện tại ra 8 biến làm việc
        a, b, c, d, e, f, g, h = self._h
        
        # Chạy 64 vòng lặp chính
        for i in range(64):
            s1 = self._sigma1(e)
            ch = self._ch(e, f, g)
            temp1 = (h + s1 + ch + self._K[i] + w[i]) & 0xFFFFFFFF
            
            s0 = self._sigma0(a)
            maj = self._maj(a, b, c)
            temp2 = (s0 + maj) & 0xFFFFFFFF
            
            # Cập nhật dòng chảy (Flow)
            h = g
            g = f
            f = e
            e = (d + temp1) & 0xFFFFFFFF
            d = c
            c = b
            b = a
            a = (temp1 + temp2) & 0xFFFFFFFF
            
        # Cộng kết quả nén vào Hash gốc (Cộng dồn)
        vals = [a, b, c, d, e, f, g, h]
        for i in range(8):
            self._h[i] = (self._h[i] + vals[i]) & 0xFFFFFFFF


    def hash(self, message: str) -> str:
        """Hàm chính: Nhận chuỗi -> Trả về mã Hex SHA-256"""
        # Reset trạng thái mỗi khi băm mới
        self._h = list(self._H_INITIAL) 
        
        # 1. Chuyển đổi và Đệm
        data_bytes = message.encode('utf-8')
        padded_data = self._pad_message(data_bytes)
        
        # 2. Xử lý từng khối 512-bit (64 bytes)
        for i in range(0, len(padded_data), 64):
            chunk = padded_data[i : i+64]
            self._compress_chunk(chunk)
            
        # 3. Xuất kết quả dạng Hex
        result = ''
        for val in self._h:
            result += '{:08x}'.format(val)
            
        return result

# --- TEST DRIVE (THỬ NGHIỆM) ---
if __name__ == "__main__":
    
    # 1. Đầu vào theo yêu cầu
    input_text = "TRANDUCLINH"
    print(f"Input: '{input_text}'")
    
    # 2. Thực hiện băm
    hasher = SHA256()
    hash_result = hasher.hash(input_text)
    
    print(f"SHA-256 Hash: {hash_result}")
    
    # 3. Kiểm tra độ dài (Phải đủ 64 ký tự hex)
    print(f"Độ dài: {len(hash_result)} ký tự")

Input: 'TRANDUCLINH'
SHA-256 Hash: 47dc67da9679f1d4027ecb46828b135faa3db6cf0b0fe98e9327a3d602cfe53b
Độ dài: 64 ký tự
